In [1]:
2

2

In [2]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))
    
    
    
from causaltune import CausalTune
from causaltune.datasets import synth_ihdp, iv_dgp_econml, generate_non_random_dataset
from causaltune.data_utils import CausalityDataset
from causaltune.dataset_processor import CausalityDatasetProcessor


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
df = synth_ihdp(return_df=True).iloc[:,:10]
df['x9'] = 'a'
df.loc[0, 'x9'] = 'b'
df.loc[1, 'x9'] = 'c'
df.loc[2, 'x9'] = 'c'
df.loc[3, 'x9'] = 'c'

df['x1'] = 'a'
df.loc[0, 'x1'] = 'b'
df.loc[1, 'x1'] = 'c'
df.loc[2, 'x1'] = 'c'
df.loc[3, 'x1'] = 'c'
df['y_factual'] = np.random.randint(0, 2, size=len(df))
df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,1,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b
1,0,1,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c
3,0,0,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c
4,0,1,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a


In [5]:
cd = CausalityDataset(data=df, treatment='treatment', outcomes=['y_factual'])

In [6]:
cd.data

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9,random
0,1,1,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b,1
1,0,1,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c,1
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c,0
3,0,0,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c,0
4,0,1,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a,1
...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,a,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0,0,a,1
743,1,1,a,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0,0,a,1
744,0,1,a,0.196818,-1.477987,0.161703,0.746189,0.467138,0,0,a,1
745,0,0,a,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1,0,a,1


In [7]:
cd.preprocess_dataset()

In [8]:
cd.data

,treatment,y_factual,random,x2,x3,x4,x5,x6,x7,x8,x1_a,x1_b,x1_c,x9_a,x9_b,x9_c
0,1,1,1.0,-0.343455,1.128554,0.161703,-0.316603,1.295216,1.0,0.0,0,1,0,0,1,0
1,0,1,1.0,-1.802002,0.383828,2.244319,-0.629189,1.295216,0.0,0.0,0,0,1,0,0,1
2,0,0,0.0,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0.0,0.0,0,0,1,0,0,1
3,0,0,0.0,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0.0,0.0,0,0,1,0,0,1
4,0,1,1.0,-0.602710,0.011465,0.161703,0.683672,-0.360940,1.0,0.0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,1.0,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0.0,0.0,1,0,0,1,0,0
743,1,1,1.0,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0.0,0.0,1,0,0,1,0,0
744,0,1,1.0,0.196818,-1.477987,0.161703,0.746189,0.467138,0.0,0.0,1,0,0,1,0,0
745,0,0,1.0,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1.0,0.0,1,0,0,1,0,0


In [9]:
df = synth_ihdp(return_df=True).iloc[:,:10]
df['x9'] = 'a'
df.loc[0, 'x9'] = 'b'
df.loc[1, 'x9'] = 'c'
df.loc[2, 'x9'] = 'c'
df.loc[3, 'x9'] = 'c'

df['x1'] = 'a'
df.loc[0, 'x1'] = 'b'
df.loc[1, 'x1'] = 'c'
df.loc[2, 'x1'] = 'c'
df.loc[3, 'x1'] = 'c'

df['y_factual'] = np.random.randint(0, 2, size=len(df))
df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,1,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b
1,0,0,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c
3,0,1,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c
4,0,0,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a


In [10]:
cd = CausalityDataset(data=df, treatment='treatment', outcomes=['y_factual'])

In [11]:
dataset_processor = CausalityDatasetProcessor()

In [12]:
dataset_processor.fit(cd, encoder_type='onehot')

CausalityDatasetProcessor()

In [13]:
b = dataset_processor

In [14]:
cd.data

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9,random
0,1,1,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b,1
1,0,0,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c,1
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c,0
3,0,1,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c,0
4,0,0,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a,1
...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,1,a,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0,0,a,0
743,1,0,a,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0,0,a,0
744,0,1,a,0.196818,-1.477987,0.161703,0.746189,0.467138,0,0,a,0
745,0,0,a,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1,0,a,1


In [15]:
cd = b.transform(cd)

In [16]:
cd.data

,treatment,y_factual,x2,x3,x4,x5,x6,random,x1_1,x1_2,x1_3,x7,x8,x9_1,x9_2,x9_3
0,1,1,-0.343455,1.128554,0.161703,-0.316603,1.295216,1.0,1,0,0,1.0,0.0,1,0,0
1,0,0,-1.802002,0.383828,2.244319,-0.629189,1.295216,1.0,0,1,0,0.0,0.0,0,1,0
2,0,0,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0.0,0,1,0,0.0,0.0,0,1,0
3,0,1,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0.0,0,1,0,0.0,0.0,0,1,0
4,0,0,-0.602710,0.011465,0.161703,0.683672,-0.360940,1.0,0,0,1,1.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,1,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0.0,0,0,1,0.0,0.0,0,0,1
743,1,0,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0.0,0,0,1,0.0,0.0,0,0,1
744,0,1,0.196818,-1.477987,0.161703,0.746189,0.467138,0.0,0,0,1,0.0,0.0,0,0,1
745,0,0,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1.0,0,0,1,1.0,0.0,0,0,1


In [17]:
df = synth_ihdp(return_df=True).iloc[:,:10]
df['x9'] = 'a'
df.loc[0, 'x9'] = 'b'
df.loc[1, 'x9'] = 'c'
df.loc[2, 'x9'] = 'c'
df.loc[3, 'x9'] = 'c'

df['x1'] = 'a'
df.loc[0, 'x1'] = 'b'
df.loc[1, 'x1'] = 'c'
df.loc[2, 'x1'] = 'c'
df.loc[3, 'x1'] = 'c'

df['y_factual'] = np.random.randint(0, 2, size=len(df))
df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,1,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b
1,0,1,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c
2,0,1,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c
3,0,0,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c
4,0,0,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a


In [18]:
cd2 = CausalityDataset(data=df, treatment='treatment', outcomes=['y_factual'])

In [19]:
cd2 = dataset_processor.transform(cd2)

In [20]:
cd2.data

,treatment,y_factual,x2,x3,x4,x5,x6,random,x1_1,x1_2,x1_3,x7,x8,x9_1,x9_2,x9_3
0,1,1,-0.343455,1.128554,0.161703,-0.316603,1.295216,0.0,1,0,0,1.0,0.0,1,0,0
1,0,1,-1.802002,0.383828,2.244319,-0.629189,1.295216,0.0,0,1,0,0.0,0.0,0,1,0
2,0,1,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,1.0,0,1,0,0.0,0.0,0,1,0
3,0,0,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0.0,0,1,0,0.0,0.0,0,1,0
4,0,0,-0.602710,0.011465,0.161703,0.683672,-0.360940,1.0,0,0,1,1.0,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0.0,0,0,1,0.0,0.0,0,0,1
743,1,1,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0.0,0,0,1,0.0,0.0,0,0,1
744,0,1,0.196818,-1.477987,0.161703,0.746189,0.467138,0.0,0,0,1,0.0,0.0,0,0,1
745,0,0,-0.202946,-1.477987,-0.879606,1.621430,0.467138,0.0,0,0,1,1.0,0.0,0,0,1


In [21]:
df = synth_ihdp(return_df=True).iloc[:,:10]
df['x9'] = 'a'
df.loc[0, 'x9'] = 'b'
df.loc[1, 'x9'] = 'c'
df.loc[2, 'x9'] = 'c'
df.loc[3, 'x9'] = 'c'

df['x1'] = 'a'
df.loc[0, 'x1'] = 'b'
df.loc[1, 'x1'] = 'c'
df.loc[2, 'x1'] = 'c'
df.loc[3, 'x1'] = 'c'

df['y_factual'] = np.random.randint(0, 2, size=len(df))
df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,0,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b
1,0,0,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c
3,0,0,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c
4,0,1,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a


In [22]:
cd = CausalityDataset(data=df, treatment='treatment', outcomes=['y_factual'])

In [23]:
# training configs

# set evaluation metric
metric = "psw_energy_distance"

# it's best to specify either time_budget or components_time_budget, 
# and let the other one be inferred; time in seconds
components_time_budget = 60

# specify training set size
train_size = 0.7

In [24]:
ct2 = CausalTune(
    estimator_list=["DomainAdaptationLearner"],
    metric=metric,
    verbose=1,
    components_time_budget=components_time_budget,
    train_size=train_size
)

In [36]:
# run causaltune
ct2.fit(data=cd, outcome=cd.outcomes[0], preprocess=True, encoder_type='woe', encoder_outcome=cd.outcomes[0])

print('---------------------')
# return best estimator
print(f"Best estimator: {ct2.best_estimator}")
# config of best estimator:
print(f"Best config: {ct2.best_config}")
# best score:
print(f"Best score: {ct2.best_score}")

[flaml.tune.tune: 06-18 19:38:51] {493} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]
[flaml.tune.tune: 06-18 19:38:51] {636} INFO - trial 1 config: {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}


Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks
Initial configs: [{'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}]
---------------------
Best estimator: backdoor.econml.metalearners.DomainAdaptationLearner
Best config: {'estimator': {'estimator_name': 'backdoor.econml.metalearners.DomainAdaptationLearner'}}
Best score: 0.13396199494334926


In [37]:
ct2.predict(cd, preprocess=True)

array([[ 0.11192536],
       [ 0.01939836],
       [-0.20387733],
       [ 0.27785403],
       [-0.20387733],
       [ 0.11192536],
       [ 0.01939836],
       [-0.20387733],
       [ 0.01939836],
       [-0.20387733],
       [ 0.11192536],
       [-0.20387733],
       [ 0.27785403],
       [ 0.11192536],
       [ 0.08447048],
       [ 0.29835856],
       [-0.20387733],
       [ 0.27785403],
       [ 0.11192536],
       [-0.20387733],
       [ 0.01939836],
       [-0.20387733],
       [ 0.01939836],
       [-0.20387733],
       [-0.20387733],
       [-0.20387733],
       [ 0.27785403],
       [ 0.37079701],
       [ 0.11192536],
       [ 0.29835856],
       [ 0.11192536],
       [-0.20387733],
       [ 0.27785403],
       [ 0.01939836],
       [ 0.11192536],
       [ 0.08447048],
       [ 0.11192536],
       [-0.20387733],
       [ 0.27785403],
       [ 0.11192536],
       [ 0.01939836],
       [ 0.27785403],
       [ 0.01939836],
       [ 0.11192536],
       [ 0.37079701],
       [-0

In [38]:
cd.data

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9,random
0,1,0,b,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,b,0
1,0,0,c,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,c,0
2,0,0,c,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,c,1
3,0,0,c,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,c,1
4,0,1,a,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,a,0
...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,a,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0,0,a,0
743,1,0,a,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0,0,a,0
744,0,1,a,0.196818,-1.477987,0.161703,0.746189,0.467138,0,0,a,1
745,0,0,a,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1,0,a,1
